In [1]:
import os

In [2]:
!pwd

/Users/kunal/Developer/DL Project/Chicken-Disease-Classification-Using-Fecal-Image/research


In [3]:
os.chdir("../") # It will change the directory i.e. it will get back one folder

In [4]:
!pwd

/Users/kunal/Developer/DL Project/Chicken-Disease-Classification-Using-Fecal-Image


In [5]:
from dataclasses import dataclass
from pathlib import Path

# Updating the entity (return type of function)
@dataclass(frozen=True) # dataclass means it not the actual python class. It is a dataclass right now & frozen=true means apart from this you can't define other variables.
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root]) # creating artifacts folder


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion # initailzing config.yaml file

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [10]:
import os
import urllib.request as request # With the help of request library I am downloading data from github
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size # get_size will give the size of the data

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL, # downloading the data from the github url
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [14]:
# Pipleline (Step of execution)
try:
    config = ConfigurationManager() # Initializing the configuration manager
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-02-28 11:02:38,313: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-02-28 11:02:38,315: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-28 11:02:38,315: INFO: common: created directory at: artifacts]
[2025-02-28 11:02:38,316: INFO: common: created directory at: artifacts/data_ingestion]
[2025-02-28 11:04:46,277: INFO: 2870809520: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 5A1C:395D0:350444:8B7D31:67C14A78
Accept-Ranges: bytes
Date: Fri, 28 Feb 2025 05:32:41 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4731-BOM
X-Cache: MI